# **Processing for Data Analysis and Report**


*I have a table of order data of about 4 million rows in Google Bigquery. In this mini project I will use python and SQL to prepare data (**collecting and data transformation**) for a Customer RFM analysis.*

*In this notebook, I use python for most of the preparation, but at the end of the notebook, I created a SQL query that does what I did in python. Because python or SQL is just another tool or approach in Data analysis processing*

## 1.Collect data from Google Bigquery

test_table Table get all of data I need for RFM analysis in this project. I will use GBQ api to collect data from my GBQ project/dataset to this notebook.

*---In fact, an RFM analysis will need more fields to get insight---*

In [89]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pandas_gbq
import numpy as np
from google.oauth2 import service_account

In [ ]:

#Credentail of my project in Google Bigquery

cre = service_account.Credentials.from_service_account_file(r'credential_js.json')

#create query and extract data from GBQ, because I will transfrom and clean data by python, so I'll query all of data (about 4m rows) from my dataset on GBQ
sql = """--sql
SELECT * FROM `mydw-383006.myDW_customer.test_table`
"""
data = pd.read_gbq(query=sql, project_id= 'mydw-383006', credentials= cre)

Extract data from GBQ take 8 minutes, Its so long on me, so I will save this data to Pickle file which easier to read!

In [72]:
data.to_pickle('data_customer')

In [73]:
data[data['AutoCode'] == 1]

,AutoCode,CreateDate,ContactPhone,ProductGroup,ProductName,Amount,Quantity


# 2.Data cleaning

In [75]:
#check datatype of this DataFrame's columns
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4697293 entries, 0 to 4697292
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype              
---  ------        --------------    -----              
 0   AutoCode      4697293 non-null  object             
 1   CreateDate    4697293 non-null  datetime64[ns, UTC]
 2   ContactPhone  4697293 non-null  object             
 3   ProductGroup  4697293 non-null  object             
 4   ProductName   4697293 non-null  object             
 5   Amount        4697293 non-null  float64            
 6   Quantity      4697293 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), object(4)
memory usage: 250.9+ MB


Fortunately, dataframe don't have mising value, but Quanity column's datatype should be int, we will convert it later. Next, I will check numeric columns in DataFrame

In [8]:
data.describe()

,Amount,Quantity
count,4.697293e+06,4.697293e+06
mean,3.210203e+06,1.331584e+00
std,7.261325e+06,8.305182e+02
min,-1.436400e+08,-5.000000e+02
25%,2.090000e+05,1.000000e+00
50%,4.800000e+05,1.000000e+00
75%,2.400000e+06,1.000000e+00
max,1.801900e+09,1.800000e+06


We can see that Amount and Quanity have negative values, it may be due to the data recognition rules of the company that owns this data. In this project, we will ignore those elements, so I will treat these orders as outlier and remove them from the dataframe.

In [77]:
df = data[(data['Amount'] > 0) & (data['Quantity'] > 0 )]
df.describe()

,Amount,Quantity
count,4.329764e+06,4.329764e+06
mean,3.629606e+06,1.033873e+00
std,7.264077e+06,1.441113e+00
min,1.000000e+00,1.000000e+00
25%,2.670000e+05,1.000000e+00
50%,5.460000e+05,1.000000e+00
75%,2.890000e+06,1.000000e+00
max,1.801900e+09,1.535000e+03


Now, I convert Quantity to Int type

In [78]:
df[(df['Quantity'] % 1) > 0]

,AutoCode,CreateDate,ContactPhone,ProductGroup,ProductName,Amount,Quantity


In [88]:
df['Quantity'] = df['Quantity'].astype('int')

In [90]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4329764 entries, 0 to 4697292
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype              
---  ------        --------------    -----              
 0   AutoCode      4329764 non-null  object             
 1   CreateDate    4329764 non-null  datetime64[ns, UTC]
 2   ContactPhone  4329764 non-null  object             
 3   ProductGroup  4329764 non-null  object             
 4   ProductName   4329764 non-null  object             
 5   Amount        4329764 non-null  int32              
 6   Quantity      4329764 non-null  int32              
dtypes: datetime64[ns, UTC](1), int32(2), object(4)
memory usage: 231.2+ MB


In [58]:
df['ContactPhone'].str.contains(r'^\d+$',regex=True).value_counts()

True     4308510
False      21254
Name: ContactPhone, dtype: int64

In [59]:
import re

def regexp_phone(str):
    patt_ = r'[^\d]'
    if re.match(string = str, pattern = r'\d+'):
        return re.sub(string = str, pattern= patt_, repl= '')
    else:
        return pd.NA


df['ContactPhone'] = df.ContactPhone.apply(regexp_phone)

C:\Users\nchib\AppData\Local\Temp\ipykernel_25328\3788047785.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ContactPhone'] = df.ContactPhone.apply(regexp_phone)


In [69]:
df

Index(['AutoCode', 'CreateDate', 'ContactPhone', 'ProductGroup', 'ProductName',
       'Amount', 'Quantity'],
      dtype='object')